In [ ]:
%load_ext autoreload
%autoreload 2

#%matplotlib inline
#%matplotlib widget
%pylab inline

import os
import sys
import cv2
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import Video as JupyterVideo

# start datajoint using local server
import datajoint as dj

# if using testing database
dj.config['database.host'] = '127.0.0.1'
dj.config['database.user'] = 'root'
dj.config['database.password'] = 'pose'

dj.config["enable_python_native_blobs"] = True

sys.path.append('..')
from pose_pipeline.pipeline import VideoSession, Video, TrackingBbox, OpenPose, CenterHMR
from pose_pipeline.pipeline import Subject, PersonBbox, PersonBboxValid, OpenPosePerson

# for openpose to work
home = os.path.expanduser("~")
openpose_python_path = os.path.join(home, 'projects/pose/openpose/build/python')
sys.path.append(openpose_python_path)

# for center HMR to work
centerhmr_python_path = os.path.join(home, 'projects/pose/CenterHMR/src')
sys.path.append(centerhmr_python_path)
sys.path.append(os.path.join(centerhmr_python_path, 'core'))

## Example analysis of data already imported

Visualize the data organization

In [ ]:
schema = dj.schema('pose_pipeline')
dj.ERD(schema)

First run the basic video analysis (can take a while, and typically will be run outside Jupyter).

In [ ]:
TrackingBbox.populate()   # this is a fairly robust person tracker through time
OpenPose.populate()       # perform bottom up keypoint detection
CenterHMR.populate()      # perform bottom up mesh regression

Check how much data is in the database

In [ ]:
display(Video())
display(TrackingBbox())
display(OpenPose())
display(CenterHMR())

## Manually annotate the person of interest

First, identify videos that haven't identified a person to track

In [ ]:
TrackingBbox - PersonBboxValid

Select one and watch it. Note the ID displayed over the person and if it changes

In [ ]:
video_filter = 'filename="sling0001_8_6"'
video = (TrackingBbox & video_filter).fetch1('output_video')
JupyterVideo(video, width=320)

In [ ]:
# note these should be manually entered and must be correct
# another data pipeline might know that "subject X" was recorded in a
# particular video session, so might suggest the right answer.
# ideally this would be implemented with a wrapper that is 
# study specific with appropriate access controls

subject_id = 'sling0001'
Subject.insert1({'subject_id': subject_id}, skip_duplicates=True)

subject_key = (Subject & {'subject_id': subject_id}).fetch1('KEY')
video_key = (TrackingBbox & video_filter).fetch1('KEY')

key = subject_key.copy()
key.update(video_key)

# IMPORTANT: this should match the (possibly multiple) IDs shown with the subject in the
# video just shown
key['keep_tracks'] = [2, 58, 98]

PersonBboxValid.insert1(key, skip_duplicates=True)

In [ ]:
# Based on this manually annotated track, extract the keypoints associated
# with that person
PersonBbox.populate(video_filter)
OpenPosePerson.populate(video_filter)

In [ ]:
video = (OpenPosePerson & video_filter).fetch1('output_video')
JupyterVideo(video, width=320)

# WIP and Notes

In [ ]:
poses_dj = CenterHMR.fetch(as_dict=True)[0]

poses = np.asarray([r['params']['body_pose'][0] for r in poses_dj['results']])

joint_idx = [0, 3]
#poses = pd.DataFrame(poses[:, joint_idx], columns=['RHip', 'RKnee'])

poses = pd.DataFrame(poses)

poses.plot()
#plt.xlim(0, 200)

In [ ]:
JupyterVideo(poses_dj['output_video'], width=640)

In [ ]:
d = OpenPose.fetch1()

In [ ]:
from pose_pipeline.deep_sort_yolov4.parser import tracking_bounding_boxes

tracks = tracking_bounding_boxes('tmpn3mr7jtt.mp4', 'out.mp4')

In [ ]:
#JupyterVideo('out.mp4')
#TrackingBbox().populate()

display(TrackingBbox())

d = (TrackingBbox & 'filename="sling0002_30_2"').fetch1()
JupyterVideo(d['output_video'], width=320)

tracks = d['tracks']

In [ ]:
d = (CenterHMR & 'filename="sling0002_30_2"').fetch1()
JupyterVideo(d['output_video'], width=480)

In [ ]:
d = (OpenPose & 'filename="sling0002_30_2"').fetch1()
JupyterVideo(d['output_video'], width=480)

In [ ]:
#tracks

def extract_person_track(tracks, keep_tracks_ids = [1, 3]):
    
    def process_timestamp(track_timestep):
        valid = [t for t in track_timestep if t['track_id'] in keep_tracks_ids]
        if len(valid) == 1:
            return {'present': True, 'bbox': valid[0]['tlwh']}
        else:
            return {'present': False, 'bbox': [0.0, 0.0, 0.0, 0.0]}
        
    return [process_timestamp(t) for t in tracks]

main_track = extract_person_track(tracks)

LD = main_track
dict_lists = {k: [dic[k] for dic in LD] for k in LD[0]}

present = np.array(dict_lists['present'])
np.array(dict_lists['bbox'])[present]
np.where(present)[0]

In [ ]:
#(TrackingBbox & 'filename="sling0002_30_0"').fetch1('tracks', 'timestamps')[1]

from pose_pipeline.pipeline import PersonBboxValid, PersonBbox

#[1, 3]

key = (TrackingBbox & 'filename="sling0002_30_2"').fetch1('KEY')
key['subject_id'] = 'sling0002'
key['keep_tracks'] = [1, 3]

PersonBboxValid.insert1(key, skip_duplicates=True)
PersonBbox.populate()

In [ ]:
bbox, frames, keypoints = (PersonBbox * OpenPose).fetch1('bbox', 'frames', 'keypoints')

In [ ]:
print(bbox[frames[0]])
keypoints[100][0]  #.astype(int)

In [ ]:
plt.figure()
plt.plot(keypoints[0][0][:, 0], keypoints[0][0][:, 1], '.')

In [ ]:

def keypoints_to_bbox(keypoints, thresh=0.1, min_keypoints=5):
    valid = keypoints[:, -1] > thresh
    keypoints = keypoints[valid, :-1]
    
    if keypoints.shape[0] < min_keypoints:
        return [0.0, 0.0, 0.0, 0.0]
    
    bbox = [np.min(keypoints[:, 0]), np.min(keypoints[:, 1]), np.max(keypoints[:, 0]), np.max(keypoints[:, 1])]
    bbox = bbox[:2] + [bbox[2] - bbox[0], bbox[3] - bbox[1]]
    
    return bbox

kp_bbox = keypoints_to_bbox(keypoints[0][0])
print(kp_bbox)

In [ ]:
def IoU(box1: np.ndarray, box2: np.ndarray, tlhw=False):
    """
    calculate intersection over union cover percent
    
        :param box1: box1 with shape (N,4)
        :param box2: box2 with shape (N,4)
        :tlhw: bool if format is tlhw and need to be converted to tlbr
        :return: IoU ratio if intersect, else 0
    """
    point_num = max(box1.shape[0], box2.shape[0])
    b1p1, b1p2, b2p1, b2p2 = box1[:, :2], box1[:, 2:], box2[:, :2], box2[:, 2:]
    
    if tlhw:
        b1p2 = b1p1 + b1p2
        b2p2 = b2p1 + b2p2   

    # mask that eliminates non-intersecting matrices
    base_mat = np.ones(shape=(point_num,)).astype(float)
    base_mat *= np.all(np.greater(b1p2 - b2p1, 0), axis=1)
    base_mat *= np.all(np.greater(b2p2 - b1p1, 0), axis=1)
    
    intersect_area = np.prod(np.minimum(b2p2, b1p2) - np.maximum(b1p1, b2p1), axis=1).astype(float)
    union_area = np.prod(b1p2 - b1p1, axis=1) + np.prod(b2p2 - b2p1, axis=1) - intersect_area
    intersect_ratio = intersect_area / union_area
    
    return base_mat * intersect_ratio

#IoU(bbox[:1], np.array(kp_bbox)[None, ...], tlhw=True)
#IoU(bbox[:1], bbox[:1], tlhw=True)
IoU(np.array([[0.75, 0.5, 1.0, 1.0]]), np.array([[0.5, 0.5, 1.0, 1.0], [0.5, 0.5, 1.0, 1.0]]) , tlhw=False)

In [ ]:
IoU(np.array([[0.75, 0.5, 1.0, 1.0]]), np.zeros((0,4)))

In [ ]:

def match_keypoints_to_bbox(bbox: np.ndarray, keypoints_list: list, thresh=0.3, num_keypoints=25):
    """ Finds the best keypoints with an acceptable IoU, if present """
    
    empty_keypoints = np.zeros((num_keypoints, 3))
    
    if len(keypoints_list) == 0:
        return empty_keypoints, None
    
    bbox = np.reshape(bbox, (1, 4))
    iou = IoU(bbox, np.array([keypoints_to_bbox(k) for k in keypoints_list]) )
    idx = np.argmax(iou)
    
    if iou[idx] > thresh:
        return keypoints_list[idx], idx
    
    return empty_keypoints, None

res = [match_keypoints_to_bbox(bbox[idx], keypoints[idx]) for idx in range(bbox.shape[0])]

In [ ]:
res2 = list(zip(*res))

In [ ]:
key = (OpenPose * PersonBboxValid & video_filter).fetch1('KEY')
keypoints = (OpenPose & key).fetch1('keypoints')
bbox = (PersonBbox & key).fetch1('bbox')

#print(keypoints)
def keypoints_to_bbox(keypoints, thresh=0.1, min_keypoints=5):
    valid = keypoints[:, -1] > thresh
    keypoints = keypoints[valid, :-1]

    if keypoints.shape[0] < min_keypoints:
        return [0.0, 0.0, 0.0, 0.0]

    bbox = [np.min(keypoints[:, 0]), np.min(keypoints[:, 1]), np.max(keypoints[:, 0]), np.max(keypoints[:, 1])]
    bbox = bbox[:2] + [bbox[2] - bbox[0], bbox[3] - bbox[1]]

    return bbox

def IoU(box1: np.ndarray, box2: np.ndarray, tlhw=False):
    """
    calculate intersection over union cover percent

        :param box1: box1 with shape (N,4)
        :param box2: box2 with shape (N,4)
        :tlhw: bool if format is tlhw and need to be converted to tlbr
        :return: IoU ratio if intersect, else 0
    """
    point_num = max(box1.shape[0], box2.shape[0])
    b1p1, b1p2, b2p1, b2p2 = box1[:, :2], box1[:, 2:], box2[:, :2], box2[:, 2:]

    if tlhw:
        b1p2 = b1p1 + b1p2
        b2p2 = b2p1 + b2p2   

    # mask that eliminates non-intersecting matrices
    base_mat = np.ones(shape=(point_num,)).astype(float)
    base_mat *= np.all(np.greater(b1p2 - b2p1, 0), axis=1)
    base_mat *= np.all(np.greater(b2p2 - b1p1, 0), axis=1)

    intersect_area = np.prod(np.minimum(b2p2, b1p2) - np.maximum(b1p1, b2p1), axis=1).astype(float)
    union_area = np.prod(b1p2 - b1p1, axis=1) + np.prod(b2p2 - b2p1, axis=1) - intersect_area
    intersect_ratio = intersect_area / union_area

    return base_mat * intersect_ratio

def match_keypoints_to_bbox(bbox: np.ndarray, keypoints_list: list, thresh=0.3, num_keypoints=25):
    """ Finds the best keypoints with an acceptable IoU, if present """

    empty_keypoints = np.zeros((num_keypoints, 3))

    if not keypoints_list or len(keypoints_list) == 0:
        return empty_keypoints, -1

    bbox = np.reshape(bbox, (1, 4))
    iou = IoU(bbox, np.array([keypoints_to_bbox(k) for k in keypoints_list]) )
    idx = np.argmax(iou)

    if iou[idx] > thresh:
        return keypoints_list[idx], idx

    return empty_keypoints, -1

print(len(keypoints))

res = [match_keypoints_to_bbox(bbox[idx], keypoints[idx]) for idx in range(bbox.shape[0])]
keypoints, openpose_ids = list(zip(*res)) 

keypoints = np.array(keypoints)
openpose_ids = np.array(openpose_ids)

print(keypoints.shape)

In [ ]:
tracks, timestamps = (TrackingBbox & key).fetch1('tracks', 'timestamps')
len(tracks)

PersonBbox.drop()

In [ ]:
key = (OpenPose * PersonBboxValid & 'filename="sling0002_30_2"').fetch1('KEY')
keypoints = (OpenPose & key).fetch1('keypoints')
bbox = (PersonBbox & key).fetch1('bbox')

print(bbox.shape)
print(len(keypoints))

In [ ]:
#video_filename = (Video & key).fetch1('video')
video_filename = (OpenPose & key).fetch1('output_video')

cap = cv2.VideoCapture(video_filename)
w, h = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = w, h = cap.get(cv2.CAP_PROP_FRAME_WIDTH), cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

ret, frame = cap.read()

dsize = (int(w // 2), int(h // 2))
print(frames)

In [ ]:
def draw_frame(frame, idx, dsize, thresh=0.25):
    frame = frame.copy()
    cv2.rectangle(frame, 
                  (int(bbox[idx, 0]), int(bbox[idx, 1])), 
                  (int(bbox[idx, 2] + bbox[idx, 0]), int(bbox[idx, 3] + bbox[idx, 1])),
                  (255, 255, 255), 15)
    for i in range(25):
        if keypoints[idx, i, -1] > thresh:
            cv2.circle(frame, (int(keypoints[idx, i, 0]), int(keypoints[idx, i, 1])), 15,
                       (255, 255, 255), -1)
            
    return cv2.resize(frame, dsize=dsize, interpolation=cv2.INTER_CUBIC)

ret, frame = cap.read()

idx = 0
frame = draw_frame(frame, idx, dsize=dsize)
plt.imshow(frame[..., ::-1])

In [ ]:
@schema
class OpenPosePerson(dj.Computed):
    definition = '''
    -> PersonBbox
    -> OpenPose
    ---
    keypoints        : longblob
    openpose_ids     : longblob
    output_video      : attach@localattach    # datajoint managed video file
    '''

    def make(self, key):

        def keypoints_to_bbox(keypoints, thresh=0.1, min_keypoints=5):
            valid = keypoints[:, -1] > thresh
            keypoints = keypoints[valid, :-1]
            
            if keypoints.shape[0] < min_keypoints:
                return [0.0, 0.0, 0.0, 0.0]
            
            bbox = [np.min(keypoints[:, 0]), np.min(keypoints[:, 1]), np.max(keypoints[:, 0]), np.max(keypoints[:, 1])]
            bbox = bbox[:2] + [bbox[2] - bbox[0], bbox[3] - bbox[1]]
            
            return bbox

        def IoU(box1: np.ndarray, box2: np.ndarray, tlhw=False):
            """
            calculate intersection over union cover percent
            
                :param box1: box1 with shape (N,4)
                :param box2: box2 with shape (N,4)
                :tlhw: bool if format is tlhw and need to be converted to tlbr
                :return: IoU ratio if intersect, else 0
            """
            point_num = max(box1.shape[0], box2.shape[0])
            b1p1, b1p2, b2p1, b2p2 = box1[:, :2], box1[:, 2:], box2[:, :2], box2[:, 2:]
            
            if tlhw:
                b1p2 = b1p1 + b1p2
                b2p2 = b2p1 + b2p2   

            # mask that eliminates non-intersecting matrices
            base_mat = np.ones(shape=(point_num,)).astype(float)
            base_mat *= np.all(np.greater(b1p2 - b2p1, 0), axis=1)
            base_mat *= np.all(np.greater(b2p2 - b1p1, 0), axis=1)
            
            intersect_area = np.prod(np.minimum(b2p2, b1p2) - np.maximum(b1p1, b2p1), axis=1).astype(float)
            union_area = np.prod(b1p2 - b1p1, axis=1) + np.prod(b2p2 - b2p1, axis=1) - intersect_area
            intersect_ratio = intersect_area / union_area
            
            return base_mat * intersect_ratio

        def match_keypoints_to_bbox(bbox: np.ndarray, keypoints_list: list, thresh=0.3, num_keypoints=25):
            """ Finds the best keypoints with an acceptable IoU, if present """
            
            empty_keypoints = np.zeros((num_keypoints, 3))
            
            if len(keypoints_list) == 0:
                return empty_keypoints, None
            
            bbox = np.reshape(bbox, (1, 4))
            iou = IoU(bbox, np.array([keypoints_to_bbox(k) for k in keypoints_list]) )
            idx = np.argmax(iou)
            
            if iou[idx] > thresh:
                return keypoints_list[idx], idx
            
            return empty_keypoints, None
            
        res = [match_keypoints_to_bbox(bbox[idx], keypoints[idx]) for idx in range(bbox.shape[0])]
        keypoints, openpose_ids = list(zip(*res)) 

        keypoints = np.array(keypoints)
        openpose_ids = np.array(openpose_ids)

        key['keypoints'] = keypoints
        key['openpose_ids'] = openpose_ids
        
        # TODO: this should probably be another object with a lot of this code generalized
        video_filename = (Video & key).fetch1('video')

        cap = cv2.VideoCapture(video_filename)
        w, h = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        fps = w, h = cap.get(cv2.CAP_PROP_FRAME_WIDTH), cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
        frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        dsize = (int(w // 2), int(h // 2))

        _, fname = tempfile.mkstemp(suffix='.mp4')
        fourcc = cv2.VideoWriter_fourcc(*'XVID')
        out = cv2.VideoWriter(fname, fourcc, fps, dsize)

        for idx in range(frames):

            def draw_frame(frame, idx=idx, dsize=dsize, thresh=0.25):
                frame = frame.copy()
                cv2.rectangle(frame, 
                            (int(bbox[idx, 0]), int(bbox[idx, 1])), 
                            (int(bbox[idx, 2] + bbox[idx, 0]), int(bbox[idx, 3] + bbox[idx, 1])),
                            (255, 255, 255), 15)
                for i in range(keypoints.shape[1]):
                    if keypoints[idx, i, -1] > thresh:
                        cv2.circle(frame, (int(keypoints[idx, i, 0]), int(keypoints[idx, i, 1])), 15,
                                (255, 255, 255), -1)
            
                return cv2.resize(frame, dsize=dsize, interpolation=cv2.INTER_CUBIC)

            ret, frame = cap.read()
            if not ret or frame is None:
                break

            outframe = draw_frame(frame)
            out.write(outframe)
        out.release()
        cap.release()

        key['output_video'] = fname
        self.insert1(key)
